In [1]:
from DDCASPT2 import DDCASPT2
import pickle, os, shutil
from glob import glob
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from math import sin, cos, pi
from tqdm.notebook import tqdm

In [2]:
# radius_range=np.arange(106,182,0.5)
radius_range=np.arange(106,182,0.25)


train_ind,test_ind=radius_range[0::2],radius_range[1::2]
# train_test_split(radius_range, test_size=0.3, random_state=0)
print(len(train_ind),len(test_ind))
with open('big_train_ind.pickle', 'wb') as handle:
    pickle.dump(train_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('big_test_ind.pickle', 'wb') as handle:
    pickle.dump(test_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('test_ind.pickle', 'rb') as handle:
#     test_ind = pickle.load(handle)

# with open('train_ind.pickle', 'rb') as handle:
#     train_ind = pickle.load(handle)
    
print(len(train_ind),len(test_ind))    

152 152
152 152


In [3]:
# for i in chains:
topdir = os.getcwd()

print(topdir)

/home/grierjones/DDCASPT2/ozone/multistate


In [4]:
def run(dirname,basis_set):
    if os.path.exists(basis_set)==False:
        os.mkdir(basis_set)
        
    # os.chdir(os.path.join(topdir,dirname))    
    for idxr, r in tqdm(enumerate(radius_range)):
        
        # Loop radius
        name=f"{dirname}_{r:.2f}"
        print(name)
        # Create files
        subdirpath = os.path.join(topdir,basis_set,f'{name}')
        if os.path.exists(subdirpath)==False:
            os.mkdir(subdirpath)
        print(subdirpath)
        # if os.path.exists(os.path.join(subdirpath,f'{name}.csv'))==False:
        shutil.rmtree(os.path.join('tmp'),name)
        
        # Write xyz
        with open(os.path.join(subdirpath,f'{name}.xyz'),'w') as f:
            f.write(f'{3}\n\n')
            
            radius=1.278
            f.write(f"""O {radius*cos(((float(r)/2)*(pi/180))):>8f} {radius*sin((float(r)/2)*(pi/180)):>8f} {0.0000:>8f}
O {0:>8f} {0:>8f} {0:>8f}
O {radius*cos(-(float(r)/2)*(pi/180)):>8f} {radius*sin(-(float(r)/2)*(pi/180)):>8f} {0:>8f}
""")

        
        if idxr>0 and os.path.join(topdir,basis_set,f'{dirname}_{r:.2f}',f"{dirname}_{r:.2f}_1.csv"):
            print(os.path.join(topdir,basis_set,f'{dirname}_{r:.2f}',f"{dirname}_{r:.2f}_1.csv"))
            # scfprevious=None
            # d = DDCASPT2(subdirpath,basis_set,name,10,14,None,scf_previous=scfprevious,casscf_previous=scfprevious,symmetry=1,spin=0,UHF=False,CIROOT="3 3 1",frozen=0,pt2maxiter=50,MSroots=3,charge=0)(inputwrite=True,run=True,feat=True)
        # else:            
            scfprevious=os.path.join(topdir,basis_set,f'{dirname}_{radius_range[idxr-1]:.2f}',f"{dirname}_{radius_range[idxr-1]:.2f}.ScfOrb")
            casscf_previous=os.path.join(topdir,basis_set,f'{dirname}_{radius_range[idxr-1]:.2f}',f"{dirname}_{radius_range[idxr-1]:.2f}.RasOrb")
            d = DDCASPT2(subdirpath,basis_set,name,4,3,10,scf_previous=scfprevious,casscf_previous=casscf_previous,symmetry=1,spin=0,UHF=False,CIROOT="3 3 1",frozen=0,pt2maxiter=50,MSroots=3,charge=0)(inputwrite=True,run=True,feat=True)
    
    for idxr, r in enumerate(radius_range):
        # Loop radius
        name=f"{dirname}_{r:.2f}"
        
        subdirpath = os.path.join(topdir,basis_set,f'{name}')
        
        
        for j in glob(os.path.join(subdirpath,"*GMJ*.csv"))+glob(os.path.join(subdirpath,"*Orb*"))+glob(os.path.join(subdirpath,"*h5"))+glob(os.path.join(subdirpath,"xmldump")):
            os.remove(j)        
    

In [5]:
# run('ozone','ANO-RCC-MB')

In [ ]:
run('ozone','ANO-RCC-VTZP')

0it [00:00, ?it/s]

ozone_106.00
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_106.00
ozone_106.25
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_106.25
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_106.25/ozone_106.25_1.csv
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/
{'I1': 0, 'I2': 1, 'I3': 2, 'I4': 3, 'I5': 4, 'I6': 5, 'I7': 6, 'I8': 7, 'I9': 8, 'I10': 9, 'A1': 10, 'A2': 11, 'A3': 12, 'S1': 13, 'S2': 14, 'S3': 15, 'S4': 16, 'S5': 17, 'S6': 18, 'S7': 19, 'S8': 20, 'S9': 21, 'S10': 22, 'S11': 23, 'S12': 24, 'S13': 25, 'S14': 26, 'S15': 27, 'S16': 28, 'S17': 29, 'S18': 30, 'S19': 31, 'S20': 32, 'S21': 33, 'S22': 34, 'S23': 35, 'S24': 36, 'S25': 37, 'S26': 38, 'S27': 39, 'S28': 40, 'S29': 41, 'S30': 42, 'S31': 43, 'S32': 44, 'S33': 45, 'S34': 46, 'S35': 47, 'S36': 48, 'S37': 49, 'S38': 50, 'S39': 51, 'S40': 52, 'S41': 53, 'S42': 54, 'S43': 55, 'S44': 56, 'S45': 57, 'S46': 58, 'S47': 59, 


Root:   0%|                                                                                                                                                              | 0/3 [00:00<?, ?it/s]

Pairs:   0%|                                                                                                                                                            | 0/12 [00:00<?, ?it/s]

Pairs:   8%|████████████▎                                                                                                                                       | 1/12 [00:01<00:13,  1.25s/it]

Pairs:  17%|████████████████████████▋                                                                                                                           | 2/12 [00:01<00:05,  1.71it/s]

Pairs:  25%|█████████████████████████████████████                                                                                                               | 3/12 [00:01<00:03,  2.46it/s]

Pairs:  33%|██████████████████████

ozone_106.50
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_106.50
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_106.50/ozone_106.50_1.csv
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/
{'I1': 0, 'I2': 1, 'I3': 2, 'I4': 3, 'I5': 4, 'I6': 5, 'I7': 6, 'I8': 7, 'I9': 8, 'I10': 9, 'A1': 10, 'A2': 11, 'A3': 12, 'S1': 13, 'S2': 14, 'S3': 15, 'S4': 16, 'S5': 17, 'S6': 18, 'S7': 19, 'S8': 20, 'S9': 21, 'S10': 22, 'S11': 23, 'S12': 24, 'S13': 25, 'S14': 26, 'S15': 27, 'S16': 28, 'S17': 29, 'S18': 30, 'S19': 31, 'S20': 32, 'S21': 33, 'S22': 34, 'S23': 35, 'S24': 36, 'S25': 37, 'S26': 38, 'S27': 39, 'S28': 40, 'S29': 41, 'S30': 42, 'S31': 43, 'S32': 44, 'S33': 45, 'S34': 46, 'S35': 47, 'S36': 48, 'S37': 49, 'S38': 50, 'S39': 51, 'S40': 52, 'S41': 53, 'S42': 54, 'S43': 55, 'S44': 56, 'S45': 57, 'S46': 58, 'S47': 59, 'S48': 60, 'S49': 61, 'S50': 62, 'S51': 63, 'S52': 64, 'S53': 65, 'S54': 66, 'S55'


Root:   0%|                                                                                                                                                              | 0/3 [00:00<?, ?it/s]

Pairs:   0%|                                                                                                                                                            | 0/12 [00:00<?, ?it/s]

Pairs:   8%|████████████▎                                                                                                                                       | 1/12 [00:01<00:14,  1.28s/it]

Pairs:  17%|████████████████████████▋                                                                                                                           | 2/12 [00:01<00:06,  1.60it/s]

Pairs:  25%|█████████████████████████████████████                                                                                                               | 3/12 [00:01<00:03,  2.43it/s]

Pairs:  33%|██████████████████████

...


Root:   0%|                                                                                                                                                              | 0/3 [00:00<?, ?it/s]

Pairs:   0%|                                                                                                                                                            | 0/12 [00:00<?, ?it/s]

Pairs:   8%|████████████▎                                                                                                                                       | 1/12 [00:01<00:13,  1.25s/it]

Pairs:  17%|████████████████████████▋                                                                                                                           | 2/12 [00:01<00:05,  1.71it/s]

Pairs:  25%|█████████████████████████████████████                                                                                                               | 3/12 [00:01<00:03,  2.44it/s]

Pairs:  33%|██████████████████████

ozone_108.25
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_108.25
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_108.25/ozone_108.25_1.csv
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/
{'I1': 0, 'I2': 1, 'I3': 2, 'I4': 3, 'I5': 4, 'I6': 5, 'I7': 6, 'I8': 7, 'I9': 8, 'I10': 9, 'A1': 10, 'A2': 11, 'A3': 12, 'S1': 13, 'S2': 14, 'S3': 15, 'S4': 16, 'S5': 17, 'S6': 18, 'S7': 19, 'S8': 20, 'S9': 21, 'S10': 22, 'S11': 23, 'S12': 24, 'S13': 25, 'S14': 26, 'S15': 27, 'S16': 28, 'S17': 29, 'S18': 30, 'S19': 31, 'S20': 32, 'S21': 33, 'S22': 34, 'S23': 35, 'S24': 36, 'S25': 37, 'S26': 38, 'S27': 39, 'S28': 40, 'S29': 41, 'S30': 42, 'S31': 43, 'S32': 44, 'S33': 45, 'S34': 46, 'S35': 47, 'S36': 48, 'S37': 49, 'S38': 50, 'S39': 51, 'S40': 52, 'S41': 53, 'S42': 54, 'S43': 55, 'S44': 56, 'S45': 57, 'S46': 58, 'S47': 59, 'S48': 60, 'S49': 61, 'S50': 62, 'S51': 63, 'S52': 64, 'S53': 65, 'S54': 66, 'S55'


Root:   0%|                                                                                                                                                              | 0/3 [00:00<?, ?it/s]

Pairs:   0%|                                                                                                                                                            | 0/12 [00:00<?, ?it/s]

Pairs:   8%|████████████▎                                                                                                                                       | 1/12 [00:01<00:13,  1.25s/it]

Pairs:  17%|████████████████████████▋                                                                                                                           | 2/12 [00:01<00:05,  1.70it/s]

Pairs:  25%|█████████████████████████████████████                                                                                                               | 3/12 [00:01<00:03,  2.45it/s]

Pairs:  33%|██████████████████████

ozone_108.50
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_108.50
/home/grierjones/DDCASPT2/ozone/multistate/ANO-RCC-VTZP/ozone_108.50/ozone_108.50_1.csv
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp/
{'I1': 0, 'I2': 1, 'I3': 2, 'I4': 3, 'I5': 4, 'I6': 5, 'I7': 6, 'I8': 7, 'I9': 8, 'I10': 9, 'A1': 10, 'A2': 11, 'A3': 12, 'S1': 13, 'S2': 14, 'S3': 15, 'S4': 16, 'S5': 17, 'S6': 18, 'S7': 19, 'S8': 20, 'S9': 21, 'S10': 22, 'S11': 23, 'S12': 24, 'S13': 25, 'S14': 26, 'S15': 27, 'S16': 28, 'S17': 29, 'S18': 30, 'S19': 31, 'S20': 32, 'S21': 33, 'S22': 34, 'S23': 35, 'S24': 36, 'S25': 37, 'S26': 38, 'S27': 39, 'S28': 40, 'S29': 41, 'S30': 42, 'S31': 43, 'S32': 44, 'S33': 45, 'S34': 46, 'S35': 47, 'S36': 48, 'S37': 49, 'S38': 50, 'S39': 51, 'S40': 52, 'S41': 53, 'S42': 54, 'S43': 55, 'S44': 56, 'S45': 57, 'S46': 58, 'S47': 59, 'S48': 60, 'S49': 61, 'S50': 62, 'S51': 63, 'S52': 64, 'S53': 65, 'S54': 66, 'S55'


Root:   0%|                                                                                                                                                              | 0/3 [00:00<?, ?it/s]

Pairs:   0%|                                                                                                                                                            | 0/12 [00:00<?, ?it/s]

Pairs:   8%|████████████▎                                                                                                                                       | 1/12 [00:01<00:13,  1.26s/it]

Pairs:  17%|████████████████████████▋                                                                                                                           | 2/12 [00:01<00:05,  1.70it/s]

Pairs:  25%|█████████████████████████████████████                                                                                                               | 3/12 [00:01<00:03,  2.58it/s]

Pairs:  33%|██████████████████████